In [1]:
from data_processor_lib import Processor

data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

In [2]:
processor = Processor()
processor.read_csv(data_src)

Profiler initialized


In [3]:
processor.columns()

Index(['Name', 'Team', 'Number', 'Position', 'Age', 'Height', 'Weight',
       'College', 'Salary'],
      dtype='object')

In [4]:
processor.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [5]:
processor.overview()

,Value
Total Columns,9.00
Total Rows,458.00
Incomplete Rows Count,94.00
Incomplete Rows (%),20.52
Incomplete Columns Count,9.00
Total Data Count,4018.00
Missing Data Count,104.00
Missing Data (%),2.59


In [6]:
processor.insights()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
Valid Data Count,457.0,457.0,457.0,457.0,457.0,457.0,457.0,373.0,446.0
Missing Data Count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,85.0,12.0
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,22.79,2.69
Data Type,object,object,float64,object,float64,object,float64,object,float64
Mean,-,-,17.68,-,26.94,-,221.52,-,4842684.11
Median,-,-,13.0,-,26.0,-,220.0,-,2839073.0
Min,-,-,0.0,-,19.0,-,161.0,-,30888.0
Max,-,-,99.0,-,40.0,-,307.0,-,25000000.0
Standard Deviation,-,-,15.97,-,4.4,-,26.37,-,5229237.6
25% Quantile,-,-,5.0,-,24.0,-,200.0,-,1044792.25


In [7]:
processor.extract_incompletes()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0
5,Amir Johnson,Boston Celtics,90.0,PF,29.0,6-9,240.0,NaN,12000000.0
15,Bojan Bogdanovic,Brooklyn Nets,44.0,SG,27.0,6-8,216.0,NaN,3425510.0
20,Sergey Karasev,Brooklyn Nets,10.0,SG,22.0,6-7,208.0,NaN,1599840.0
...,...,...,...,...,...,...,...,...,...
447,Rudy Gobert,Utah Jazz,27.0,C,23.0,7-1,245.0,NaN,1175880.0
450,Joe Ingles,Utah Jazz,2.0,SF,28.0,6-8,226.0,NaN,2050000.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0


In [8]:
processor.set_ignore_rows([457])
processor.set_ignore_columns(['College'])

In [9]:
processor.extract_incompletes()

,Number,Age,Salary,Name,Weight,Position,Team,Height
2,30.0,27.0,NaN,John Holland,205.0,SG,Boston Celtics,6-5
46,42.0,37.0,NaN,Elton Brand,254.0,PF,Philadelphia 76ers,6-9
171,30.0,35.0,NaN,Dahntay Jones,225.0,SG,Cleveland Cavaliers,6-6
264,4.0,29.0,NaN,Jordan Farmar,180.0,PG,Memphis Grizzlies,6-2
269,5.0,24.0,NaN,Ray McCallum,190.0,PG,Memphis Grizzlies,6-3
270,14.0,24.0,NaN,Xavier Munford,180.0,PG,Memphis Grizzlies,6-3
273,35.0,28.0,NaN,Alex Stepheson,270.0,PF,Memphis Grizzlies,6-10
350,12.0,23.0,NaN,Briante Weber,165.0,PG,Miami Heat,6-2
353,11.0,30.0,NaN,Dorell Wright,205.0,SF,Miami Heat,6-9
397,6.0,23.0,NaN,Axel Toupane,210.0,SG,Denver Nuggets,6-7


In [10]:
processor.overview()

,Value
Total Columns,8.00
Total Rows,457.00
Incomplete Rows Count,11.00
Incomplete Rows (%),2.41
Incomplete Columns Count,8.00
Total Data Count,3645.00
Missing Data Count,11.00
Missing Data (%),0.30


In [11]:
processor.insights()

,Number,Age,Salary,Name,Weight,Position,Team,Height
Valid Data Count,457.0,457.0,446.0,457.0,457.0,457.0,457.0,457.0
Missing Data Count,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0
Missing Data (%),0.0,0.0,2.47,0.0,0.0,0.0,0.0,0.0
Data Type,float64,float64,float64,object,float64,object,object,object
Mean,17.68,26.94,4842684.11,-,221.52,-,-,-
Median,13.0,26.0,2839073.0,-,220.0,-,-,-
Min,0.0,19.0,30888.0,-,161.0,-,-,-
Max,99.0,40.0,25000000.0,-,307.0,-,-,-
Standard Deviation,15.97,4.4,5229237.6,-,26.37,-,-,-
25% Quantile,5.0,24.0,1044792.25,-,200.0,-,-,-


In [12]:
processor.set_sample(frac=0.5)
processor.sample

,Number,Age,Salary,Name,Weight,Position,Team,Height
0,0.0,25.0,7730337.0,Avery Bradley,180.0,PG,Boston Celtics,6-2
1,99.0,25.0,6796117.0,Jae Crowder,235.0,SF,Boston Celtics,6-6
5,90.0,29.0,12000000.0,Amir Johnson,240.0,PF,Boston Celtics,6-9
8,12.0,22.0,1824360.0,Terry Rozier,190.0,PG,Boston Celtics,6-2
10,7.0,24.0,2569260.0,Jared Sullinger,260.0,C,Boston Celtics,6-9
...,...,...,...,...,...,...,...,...
450,2.0,28.0,2050000.0,Joe Ingles,226.0,SF,Utah Jazz,6-8
451,23.0,26.0,981348.0,Chris Johnson,206.0,SF,Utah Jazz,6-6
453,8.0,26.0,2433333.0,Shelvin Mack,203.0,PG,Utah Jazz,6-3
454,25.0,24.0,900000.0,Raul Neto,179.0,PG,Utah Jazz,6-1


In [13]:
processor.sample_insights()

,Number,Age,Salary,Name,Weight,Position,Team,Height
Valid Data Count,228.0,228.0,224.0,228.0,228.0,228.0,228.0,228.0
Missing Data Count,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
Missing Data (%),0.0,0.0,1.79,0.0,0.0,0.0,0.0,0.0
Data Type,float64,float64,float64,object,float64,object,object,object
Mean,16.74,27.02,4846006.92,-,221.31,-,-,-
Median,13.0,26.0,2839073.0,-,220.0,-,-,-
Min,0.0,19.0,55722.0,-,161.0,-,-,-
Max,99.0,40.0,22359364.0,-,289.0,-,-,-
Standard Deviation,15.61,4.44,5233984.42,-,27.17,-,-,-
25% Quantile,5.0,24.0,1000000.0,-,200.0,-,-,-


In [14]:
processor.sample_overview()

,Value
Total Columns,8.00
Total Rows,228.00
Incomplete Rows Count,4.00
Incomplete Rows (%),1.75
Incomplete Columns Count,8.00
Total Data Count,1820.00
Missing Data Count,4.00
Missing Data (%),0.22
